# Init

In [1]:
!sage -pip install -q plotly tqdm

In [2]:
from tqdm.auto import tqdm
import plotly.express as px

In [3]:
%display latex

# https://github.com/jupyter-lsp/jupyterlab-lsp/issues/910#issuecomment-1465356625

# Backprojection

## Extrinsics

In [4]:
f1, f2 = var("f1 f2")
F = diagonal_matrix([f1, f2, 1])

In [5]:
RT = matrix(SR, [[f"R_{i}{j}" for i in range(1, 4)] + [f"t_{j}"] for j in range(1, 4)])

In [6]:
P = F * RT

In [7]:
F, RT, P

(
[f1  0  0]  [R_11 R_21 R_31  t_1]  [R_11*f1 R_21*f1 R_31*f1  f1*t_1]
[ 0 f2  0]  [R_12 R_22 R_32  t_2]  [R_12*f2 R_22*f2 R_32*f2  f2*t_2]
[ 0  0  1], [R_13 R_23 R_33  t_3], [   R_13    R_23    R_33     t_3]
)

In [8]:
# [a == b for a, b in zip((F*F.T).list() , (RT*RT.T).list())]

## Intrinsics

In [9]:
alpha, s, u1, u2 = var("alpha s u1 u2")
u = vector([u1, u2, 1])
e = vector(list(var("e_x e_y")) + [1])
T = matrix.identity(SR, 3)
T[:, 2] = -e

In [10]:
A = diagonal_matrix([1 / alpha, 1, 1]) * diagonal_matrix([1 / s, 1 / s, 1]) * T
A

[   1/(alpha*s)              0 -e_x/(alpha*s)]
[             0            1/s         -e_y/s]
[             0              0             -1]

## 3D point

In [11]:
X = vector(SR, [var(f"X_{i}") for i in range(1, 3)] + [0, 1])
X

(X_1, X_2, 0, 1)

## Distortion

In [12]:
lambda_1, lambda_2, lambda_3 = var("lambda_1 lambda_2 lambda_3", domain="real")


def psi(r, n=2):
    assert 1 <= n <= 3
    ret = 1 + lambda_1 * r ^ 2
    if n >= 2:
        ret += lambda_2 * r ^ 4
    if n == 3:
        ret += lambda_3 * r ^ 6
    return ret


r = lambda u: norm(u[0:2])
g = lambda u, n=2: vector([u[0], u[1], psi(r(u), n)])

In [13]:
g(A * u)

(-e_x/(alpha*s) + u1/(alpha*s), -e_y/s + u2/s, (abs(-e_y/s + u2/s)^2 + abs(-e_x/(alpha*s) + u1/(alpha*s))^2)^2*lambda_2 + (abs(-e_y/s + u2/s)^2 + abs(-e_x/(alpha*s) + u1/(alpha*s))^2)*lambda_1 + 1)

## Backprojection

In [14]:
gamma = var("gamma")

In [15]:
# https://github.com/sagemath/sage/issues/35264
backproj = gamma * g(A * u), P * X

display(backproj[0])
display(backproj[1])

(-gamma*(e_x/(alpha*s) - u1/(alpha*s)), -gamma*(e_y/s - u2/s), ((abs(-e_y/s + u2/s)^2 + abs(-e_x/(alpha*s) + u1/(alpha*s))^2)^2*lambda_2 + (abs(-e_y/s + u2/s)^2 + abs(-e_x/(alpha*s) + u1/(alpha*s))^2)*lambda_1 + 1)*gamma)

(R_11*X_1*f1 + R_21*X_2*f1 + f1*t_1, R_12*X_1*f2 + R_22*X_2*f2 + f2*t_2, R_13*X_1 + R_23*X_2 + t_3)

In [16]:
for lhs, rhs in zip(gamma * g(A * u), P * X):
    display(lhs == rhs)

-gamma*(e_x/(alpha*s) - u1/(alpha*s)) == R_11*X_1*f1 + R_21*X_2*f1 + f1*t_1

-gamma*(e_y/s - u2/s) == R_12*X_1*f2 + R_22*X_2*f2 + f2*t_2

((abs(-e_y/s + u2/s)^2 + abs(-e_x/(alpha*s) + u1/(alpha*s))^2)^2*lambda_2 + (abs(-e_y/s + u2/s)^2 + abs(-e_x/(alpha*s) + u1/(alpha*s))^2)*lambda_1 + 1)*gamma == R_13*X_1 + R_23*X_2 + t_3